# K-Fold

เป็นการทำ Cross Validation ไม่ใช่เพื่อหา model ที่ดีที่สุด แต่เป็นการทำเพื่อดูความ generalize ของมัน ว่าหากไปเจอ Data ที่ไม่เคยเจอมาก่อน มันจะ perform ได้ดีแค่ไหน    

ระดับ Basic คือการ startify K-Fold คือการแบ่งข้อมูลเป็นเซ็ต เท่าๆกัน

In [10]:
from sklearn.model_selection import KFold
import yfinance as yf
import pandas as pd
import numpy as np

In [8]:
kfold = KFold(3, shuffle= True, random_state= 42) # random_state คือpattern การ shuffle เราจะได้เรียกข้อมูลที่มันสลับ มาให้ได้เป็นแบบเดิมได้
# argument แรก คือcv ว่าจะแบ่งเป็นกี่ส่วน

In [9]:
kfold

KFold(n_splits=3, random_state=42, shuffle=True)

In [11]:
start = '2017-01-01'
end = '2021-11-01'

In [12]:
data = yf.download('CL=F PTTEP.BK GPSC.BK PTTGC.BK PTT.BK', start= start, end= end)

[*********************100%***********************]  5 of 5 completed


In [15]:
df = data['Adj Close']

In [17]:
df = df.pct_change()

In [24]:
df = df.dropna()

In [25]:
df = df[['CL=F', 'GPSC.BK', 'PTTEP.BK', 'PTTGC.BK', 'PTT.BK']]
# จัดเรียง Cols ใหม่ ให้ PTT.BK เป็นy (y จะเป็น col สุดท้าย และทั้งหมดก่อนหน้าจะเป็น x)

In [26]:
df

,CL=F,GPSC.BK,PTTEP.BK,PTTGC.BK,PTT.BK
Date,,,,,
2017-01-05,0.009388,-0.020270,-0.005076,0.003906,0.018421
2017-01-06,0.004278,0.000000,0.005102,-0.003891,0.005168
2017-01-09,-0.037600,0.000000,-0.007614,-0.011719,-0.015424
2017-01-10,-0.021940,0.006897,-0.017903,0.003953,0.013055
2017-01-11,0.028139,0.006849,0.005208,-0.003937,-0.007732
...,...,...,...,...,...
2021-10-25,0.000000,-0.003247,-0.023904,-0.011364,-0.012500
2021-10-26,0.010626,0.013029,0.012245,-0.003831,-0.006329
2021-10-27,-0.023509,-0.003215,-0.020161,-0.019231,-0.012739


In [28]:
X = df.iloc[:,:-1]
y = df.iloc[:, -1]

In [29]:
for train, test in kfold.split(y):
    print('train: %s, test: %s' % (y[train], y[test]))

# data จะแยกมาใหเป็นจำนวนชุด ตามที่แบ่งไว้

train: Date
2017-01-05    0.018421
2017-01-06    0.005168
2017-01-11   -0.007732
2017-01-16    0.015666
2017-01-17   -0.005142
                ...   
2021-10-25   -0.012500
2021-10-26   -0.006329
2021-10-27   -0.012739
2021-10-28   -0.012903
2021-10-29   -0.006536
Name: PTT.BK, Length: 833, dtype: float64, test: Date
2017-01-09   -0.015424
2017-01-10    0.013055
2017-01-12   -0.005195
2017-01-13    0.000000
2017-01-19    0.000000
                ...   
2021-10-11    0.006289
2021-10-13    0.000000
2021-10-15   -0.006329
2021-10-18    0.019108
2021-10-19   -0.012500
Name: PTT.BK, Length: 417, dtype: float64
train: Date
2017-01-06    0.005168
2017-01-09   -0.015424
2017-01-10    0.013055
2017-01-11   -0.007732
2017-01-12   -0.005195
                ...   
2021-10-19   -0.012500
2021-10-20    0.000000
2021-10-25   -0.012500
2021-10-27   -0.012739
2021-10-29   -0.006536
Name: PTT.BK, Length: 833, dtype: float64, test: Date
2017-01-05    0.018421
2017-01-16    0.015666
2017-01-18   -0.00516

In [40]:
# prepared Train and Test data to LM and CV model 
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size= 0.2, random_state= 12)

In [45]:
##### ทำจริงใช้แบบนี้ แบบข้างบนๆ ไม่ได้ใช้ #####
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LinearRegression

lm = LinearRegression()
scores = cross_val_score(lm, X_train, y_train, scoring= 'r2', cv=kfold)
scores
# เอาเข้า model และ เทรนเทสเพื่อดู r2


array([0.64227551, 0.59771834, 0.71074655])

In [50]:
scores_mse = cross_val_score(lm, X_train, y_train, scoring= 'neg_mean_squared_error')
scores_mse

array([-1.08611459e-04, -8.84662740e-05, -8.72668465e-05, -1.20725161e-04,
       -1.00927304e-04])

In [53]:
%%timeit
scores_mse = cross_val_score(lm, X_train, y_train, scoring= 'neg_mean_squared_error', n_jobs=-1) # n_job -1 คือใช้ทุก procressor
scores_mse

10.5 ms ± 425 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)
